In [1]:
import logging
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from dotenv import load_dotenv
from datetime import datetime, timedelta
import json
import os
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from datetime import datetime, timedelta
from datetime import datetime, date, timedelta
import json
import logging
import os

from caldav import DAVClient
from dotenv import load_dotenv
import icalendar
import sys



# Configuration simple
CALDAV_USER = os.getenv("CALDAV_USER")
CALDAV_PASSWORD = os.getenv("CALDAV_PASSWORD")
CALDAV_URL = 'https://caldav.icloud.com'
CALENDAR_NAME = "Family"
SEARCH_TERMS = ['🏓', '🎾']

email = os.getenv('YOUR_SECRET_EMAIL')
password = os.getenv('YOUR_SECRET_PASSWORD')
logon_url = os.getenv("YOUR_SECRET_LOGON_URL")
planning_url = os.getenv('YOUR_SECRET_PLANNING_URL')
login_url = os.getenv('YOUR_SECRET_LOGIN_URL')
my_name = os.getenv('YOUR_SECRET_MY_NAME')

website_appointment =os.getenv("YOUR_SECRET_APPOINTMENT")
email_from = os.environ.get('EMAIL_FROM')
email_to = os.environ.get('EMAIL_TO')
email_password = os.environ.get('EMAIL_PASSWORD')
smtp_server = os.environ.get('SMTP_SERVER', 'smtp.gmail.com')
smtp_port = int(os.environ.get('SMTP_PORT', '587'))

# Configuration simple
CALDAV_USER = os.getenv("CALDAV_USER")
CALDAV_PASSWORD = os.getenv("CALDAV_PASSWORD")
CALDAV_URL = 'https://caldav.icloud.com'
CALENDAR_NAME = "Family"
SEARCH_TERMS = ['🏓', '🎾']


# Configure logger
def setup_logger(debug_mode=False):
    level = logging.DEBUG if debug_mode else logging.INFO
    
    logging.basicConfig(
        level=level,
        format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
        handlers=[
            logging.StreamHandler(sys.stdout)  # Print to console
        ]
    )
    return logging.getLogger(__name__)


# Initialize logger
DEBUG = True  # Set to False in production
logger = setup_logger(debug_mode=DEBUG)
time_sleep=5


# Vérification des secrets
logger.info("=== STATUS DES AUTRES SECRETS (MASQUÉS) ===")
logger.info(f"EMAIL: {'✅ DÉFINI' if email else '❌ MISSING'}")
logger.info(f"PASSWORD: {'✅ DÉFINI' if password else '❌ MISSING'}")
logger.info(f"MY_NAME: {'✅ DÉFINI' if my_name else '❌ MISSING'}")
logger.info(f"LOGON_URL: {'✅ DÉFINI' if logon_url else '❌ MISSING'}")
logger.info(f"PLANNING_URL: {'✅ DÉFINI' if planning_url else '❌ MISSING'}")
logger.info(f"LOGIN_URL: {'✅ DÉFINI' if login_url else '❌ MISSING'}")
logger.info(f"website_appointment: {'✅ DÉFINI' if website_appointment else '❌ MISSING'}")
logger.info(f"email_from: {'✅ DÉFINI' if email_from else '❌ MISSING'}")
logger.info(f"email_to: {'✅ DÉFINI' if email_to else '❌ MISSING'}")
logger.info(f"email_password: {'✅ DÉFINI' if email_password else '❌ MISSING'}")
logger.info(f"CALDAV_USER: {'✅ DÉFINI' if CALDAV_USER else '❌ MISSING'}")
logger.info(f"CALDAV_PASSWORD: {'✅ DÉFINI' if CALDAV_PASSWORD else '❌ MISSING'}")


# Vérification complète
secrets_dict = {
    "EMAIL": email,
    "PASSWORD": password,
    "MY_NAME": my_name,
    "LOGON_URL": logon_url,
    "PLANNING_URL": planning_url,
    "LOGIN_URL": login_url,
    "website_appointment": website_appointment,
    "email_to":email_to,
    "email_from":email_from,
    "email_password":email_password,
    "CALDAV_USER":CALDAV_USER,
    "CALDAV_PASSWORD":CALDAV_PASSWORD
}

missing_secrets = [name for name, value in secrets_dict.items() if not value]

if missing_secrets:
    logger.error("❌ ERREUR: UN OU PLUSIEURS SECRETS SONT MANQUANTS")
    logger.error("=== SECRETS MANQUANTS ===")
    for secret_name in missing_secrets:
        logger.error(f"{secret_name}: ❌ MISSING")
    logger.critical("ARRÊT DU PROGRAMME - VÉRIFIEZ VOS VARIABLES D'ENVIRONNEMENT")
    sys.exit(1)
else:
    logger.info("✅ TOUS LES SECRETS SONT DÉFINIS")
    logger.debug("Poursuite de l'exécution...")


class SecureWebLogin:
    """Classe pour gérer le login sécurisé avec driver comme objet"""
    
    def __init__(self, website_url, email_login, secret_password):
        self.website_url = website_url
        self.email_login = email_login
        self.secret_password = secret_password
        self.driver = None
        self._setup_driver()
    
    def _setup_driver(self):
        """Initialise le driver Chrome"""
        logger.debug("Initialisation du driver Chrome...")
        # Configuration
        options = Options()
        options.add_argument('--headless')
        options.add_argument('--no-sandbox')
        
        # Driver
        service = Service(ChromeDriverManager().install())
        self.driver = webdriver.Chrome(service=service, options=options)
        
        # Si vous avez déjà un driver ouvert
        #self.driver.set_window_size(1920, 1080)
        #self.driver.maximize_window()
        # Ou définir une taille spécifique

        logger.debug("Driver configuré avec succès")
    
    def login(self):
        """Script simple : saisir email + password et prendre screenshots"""
        logger.info("🚀 Script simple - Email + Password + Screenshots")
        logger.info(f"🌐 Site: {self.website_url}")

        self.driver.get(self.website_url)
        self.driver.set_window_size(1400, 900)
        logger.info(f"📄 Titre: {self.driver.title}")

        wait = WebDriverWait(self.driver, time_sleep)  # attend max 15s
        try:
            # 🕐 Attendre que le champ email soit présent ET visible
            email_field = wait.until(EC.visibility_of_element_located((By.XPATH, "//input[@type='email']")))
            logger.info("✅ Champ email visible, on peut continuer")

            # Remplir email
            email_field.clear()
            email_field.send_keys(self.email_login)
            logger.info("📧 Email saisi")

            # 🕐 Attendre le champ password
            password_field = wait.until(EC.visibility_of_element_located((By.XPATH, "//input[@type='password']")))
            password_field.clear()
            password_field.send_keys(self.secret_password)
            logger.info("🔑 Password saisi")

            # 🕐 Attendre et cliquer sur le bouton Login
            login_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//input[@value='Login']")))
            login_button.click()
            logger.info("✅ Bouton 'Login' cliqué")

            # 🕐 Attendre que la redirection soit terminée (ex: titre ou URL)
            wait.until(EC.url_changes(self.website_url))
            logger.info(f"🌍 Nouvelle URL : {self.driver.current_url}")

        except TimeoutException as e:
            logger.error(f"❌ Timeout pendant le login : {e}")
            raise

        return self.driver



    def go_url(self, url):
        logger.info(f"🔗 Navigation vers {url}")
        self.driver.get(url)

    def go_planning(self,next_week_booking = True):
        self.go_url(logon_url)

        if next_week_booking:
            self.driver.find_element(By.XPATH, "//input[@value='Next Week']").click()
        return self.driver    
    def go_appointment(self, appointment_url=website_appointment):
        self.go_url(appointment_url)
        wait = WebDriverWait(self.driver, time_sleep)
        try:
            # 🕐 Attendre que la liste déroulante soit présente
            select_element = wait.until(EC.presence_of_element_located((By.ID, "Users")))
            select = Select(select_element)
            select.select_by_visible_text("Serge Lebon (Family Lebon, Sandrine)")

            # 🕐 Attendre que le bouton Search soit cliquable
            search_button = wait.until(EC.element_to_be_clickable(
                (By.XPATH, "//input[@type='submit' and @value='Search']")
            ))
            search_button.click()
            logger.debug("🔍 Bouton 'Search' cliqué")

        except TimeoutException:
            logger.error("❌ Timeout: la page appointment n’a pas chargé correctement")
            raise

        return self.driver
    
    def save_appointments_json(self):
        self.go_appointment()
        wait = WebDriverWait(self.driver, time_sleep)

        try:
            # 🕐 Attendre que le tableau soit visible
            wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, "tbody tr.align-middle")))
            rows = self.driver.find_elements(By.CSS_SELECTOR, "tbody tr.align-middle")
        except TimeoutException:
            logger.error("❌ Aucun rendez-vous trouvé (timeout)")
            rows = []

        appointments = []
        for row in rows:
            cells = row.find_elements(By.TAG_NAME, "td")
            if len(cells) >= 2:
                appointments.append({
                    "date": cells[0].text,
                    "time": cells[1].text,
                    "name": cells[4].text,
                })

        with open('appointments.json', 'w', encoding='utf-8') as f:
            json.dump(appointments, f, indent=2)

        logger.info(f"✓ {len(appointments)} appointments saved!")


    def get_driver(self):
        """Retourne le driver pour utilisation externe"""
        return self.driver


    def look_for_slots(self, target_date, target_level, target_time):
        print("🔍 Recherche des créneaux...")
        course_buttons = self.driver.find_elements(By.XPATH, "//button[@data-class-time]")
        for button in course_buttons:
            class_name = button.get_attribute('data-class-name') or ''
            class_date = button.get_attribute('data-class-date') or ''
            class_time = button.get_attribute('data-class-time') or ''
            class_spaces = button.get_attribute('data-class-spaces') or '0'
            if target_date in class_date and target_level in class_name and target_time in class_time:
                print(f"✅ Créneau trouvé : {class_name} à {class_time} ({class_spaces} places)")
                return {
                    'button': button,
                    'spaces': int(class_spaces)
                }
        print("❌ Aucun créneau correspondant trouvé.")
        return None

    def click_on_slot(self, slot_available):
        info_button = slot_available['button']
        parent = info_button.find_element(By.XPATH, "./..")
        book_buttons = parent.find_elements(By.XPATH, ".//*[contains(text(), 'Book Now') or contains(text(), 'Book')]")
        if book_buttons:
            book_buttons[0].click()
            print("✅ Créneau réservé (clic sur Book).")
            time.sleep(time_sleep)
            return True
        return False        
         
    def click_for_me(self, my_name):
        moi_elements = self.driver.find_elements(By.XPATH, f"//*[contains(text(), '{my_name}')]")
        for moi_element in moi_elements:
            parent = moi_element.find_element(By.XPATH, "./..")
            moi_buttons = parent.find_elements(By.XPATH, ".//*[contains(text(), 'Book') or contains(text(), 'Réserver')]")
            if moi_buttons:
                moi_buttons[0].click()
                logging.info(f"✅ Réservation effectuée pour {my_name}")
                return True
        return False       
    
    def click_confirm_basket(self):
        basket = self.driver.find_elements(By.XPATH, "//*[contains(text(), 'Checkout')]")
        if basket:
            basket[0].click()
            time.sleep(time_sleep)
            logger.info("✅ Panier confirmé")
            return True
        return False

        
    def quit(self):
        """Ferme le driver"""
        if self.driver:
            self.driver.quit()
            logger.info("🔒 Driver fermé")
    
    

# Usage:
# secure_login = SecureWebLogin("https://site.com", "email@test.com", "password123")
# secure_login.login()
# 
# Utiliser le driver dans d'autres fonctions:
# driver = secure_login.driver


def load_appointments(filename='appointments.json'):
    """Load appointments from JSON file"""
    try:
        with open(filename, 'r', encoding='utf-8') as f:
            appointments = json.load(f)
            logger.info(f"✓ Loaded {len(appointments)} appointments")
            logger.info(f"\n📅 Appointments to send:")
            for idx, apt in enumerate(appointments, 1):
                date = apt['date'].replace('\n', ' ')
                time = apt['time'].replace('\n', ' ')
                name = apt['name'].replace('\n', ' ')


                logger.info(f"  {idx}. {date} - {time} - {name}")
    

            return appointments
    except FileNotFoundError:
        logger.error(f"❌ File {filename} not found")
        return []
    except json.JSONDecodeError as e:
        logger.error(f"❌ Invalid JSON: {e}")
        return []

def send_all_appointments_email(appointments):
    """Send email with ALL appointments"""
    email_from = os.environ.get('EMAIL_FROM')
    email_to = os.environ.get('EMAIL_TO')
    email_password = os.environ.get('EMAIL_PASSWORD')
    smtp_server = os.environ.get('SMTP_SERVER', 'smtp.gmail.com')
    smtp_port = int(os.environ.get('SMTP_PORT', '587'))
    
    if not all([email_from, email_to, email_password]):
        logger.error("❌ Email configuration missing")
        return False
    if appointments is None:
        logger.info("no appointments")
    # Email subject

    subject = f"🏓 Your Pickleball Appointments - {len(appointments)} total"
    
    # Email body
    body = "Hello!\n\n"
    body += f"Here are all your scheduled pickleball appointments ({len(appointments)} total):\n\n"
    
    for idx, apt in enumerate(appointments, 1):
        date_clean = apt['date'].replace('\n', ' ')
        time_clean = apt['time'].replace('\n', ' ')
        body += f"{idx}. {date_clean} at {time_clean}\n"
    
    body += "\n📍 Location: Calgary JCC\n"
    body += "\nSee you on the court! 🏓\n"
    
    # Create message
    msg = MIMEMultipart()
    msg['From'] = email_from
    msg['To'] = email_to
    msg['Subject'] = subject
    msg.attach(MIMEText(body, 'plain'))
    
    # Send email
    try:
        logger.info(f"📧 Connecting to {smtp_server}:{smtp_port}...")
        server = smtplib.SMTP(smtp_server, smtp_port, timeout=10)
        server.starttls()
        logger.info(f"🔐 Logging in...")
        server.login(email_from, email_password)
        logger.info(f"📤 Sending email...")
        server.send_message(msg)
        server.quit()
        logger.info(f"✅ Email sent to {email_to}")
        return True
    except Exception as e:
        logger.error(f"❌ Failed to send email: {e}")
        return False






class PickleballCalendarManager:
    def __init__(self, calendar_name=CALENDAR_NAME):
        if not CALDAV_USER or not CALDAV_PASSWORD:
            raise ValueError("CALDAV_USER et CALDAV_PASSWORD requis dans .env")
        
        self.calendar_name = calendar_name
        self.calendar = self._connect_calendar()
    
    def _connect_calendar(self):
        client = DAVClient(
            url=CALDAV_URL,
            username=CALDAV_USER,
            password=CALDAV_PASSWORD
        )
        return client.principal().calendar(name=self.calendar_name)
    
    def find_events(self, start_date=None, days_ahead=7, search_terms=SEARCH_TERMS):
        """Trouve les événements avec les emojis recherchés"""
        if start_date is None:
            start_date = datetime.now() + timedelta(days=1)
        
        end_date = start_date + timedelta(days=days_ahead)
        
        try:
            events = self.calendar.search(start=start_date, end=end_date)
            found_events = []
            
            for event in events:
                cal = icalendar.Calendar.from_ical(event.data)
                for component in cal.walk():
                    if component.name == "VEVENT":
                        summary = str(component.get('SUMMARY', ''))
                        if any(emoji in summary for emoji in search_terms):
                            found_events.append({
                                'summary': summary,
                                'start': component.get('dtstart').dt,
                                'end': component.get('dtend').dt,
                                'uid': str(component.get('uid'))
                            })
            
            logger.info(f"🔍 Trouvé {len(found_events)} événements")
            return found_events
        except Exception as e:
            logger.error(f"❌ Erreur recherche: {e}")
            return []
    
    def delete_events(self, events):
        """Supprime une liste d'événements"""
        deleted = 0
        for event in events:
            try:
                cal_event = self.calendar.event_by_uid(event['uid'])
                cal_event.delete()
                deleted += 1
            except Exception as e:
                logger.warning(f"⚠️ Impossible de supprimer {event['uid']}: {e}")
        
        logger.info(f"🗑️ Supprimé {deleted}/{len(events)} événements")
    
    def create_events(self, events):
        """Crée une liste d'événements"""
        created = 0
        for event in events:
            try:
                cal = icalendar.Calendar()
                ical_event = icalendar.Event()
                
                ical_event.add('summary', event['summary'])
                ical_event.add('dtstart', event['start'])
                ical_event.add('dtend', event['end'])
                ical_event.add('uid', f"pickleball-{event['start'].strftime('%Y%m%d-%H%M')}")
                
                cal.add_component(ical_event)
                self.calendar.save_event(cal.to_ical())
                created += 1
                
            except Exception as e:
                logger.error(f"❌ Échec création: {e}")
        
        logger.info(f"➕ Créé {created}/{len(events)} événements")
    
    def parse_appointments_json(self, filepath="appointments.json"):
        # Mapping des mois
        MONTH_MAP = {
        'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6,
        'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12
        }
        """Parse le JSON et retourne les événements futurs"""
        try:
            with open(filepath, 'r') as f:
                appointments = json.load(f)
        except FileNotFoundError:
            logger.error(f"❌ Fichier non trouvé: {filepath}")
            return []
        except json.JSONDecodeError as e:
            logger.error(f"❌ JSON invalide: {e}")
            return []
        
        today = date.today()
        current_year = today.year
        current_month = today.month
        events = []
        
        for apt in appointments:
            try:
                # Parser la date
                date_parts = apt['date'].split('\n')
                day = int(date_parts[1])
                month = MONTH_MAP[date_parts[2]]
                name = apt['name']
                
                # Année: actuelle si mois >= actuel, sinon suivante
                year = current_year if month >= current_month else current_year + 1
                
                # Parser les heures
                time_parts = apt['time'].replace('\n', ' ').split(' - ')
                start = datetime.strptime(
                    f"{year}-{month:02d}-{day:02d} {time_parts[0].strip()}", 
                    "%Y-%m-%d %I:%M %p"
                )
                end = datetime.strptime(
                    f"{year}-{month:02d}-{day:02d} {time_parts[1].strip()}", 
                    "%Y-%m-%d %I:%M %p"
                )
                
                # Garder seulement les futurs
                if start.date() > today:
                    events.append({
                        'start': start,
                        'end': end,
                        'summary': f'🏓 Registration {name}'
                    })
                    
            except Exception as e:
                logger.warning(f"⚠️ Erreur parsing: {e}")
        
        events.sort(key=lambda x: x['start'])
        logger.info(f"📅 Parsé {len(events)} événements futurs")
        
        return events
    
    def sync_pickleball_events(self):
        """Synchronisation complète"""
        logger.info("🔄 Début synchronisation")
        logger.info("=" * 50)
        
        # Nettoyer les anciens
        old_events = self.find_events(days_ahead=60)
        if old_events:
            self.delete_events(old_events)
        
        # Créer les nouveaux
        new_events = self.parse_appointments_json()
        if new_events:
            logger.info(f"📅 Création de {len(new_events)} événements:")
            for event in new_events:  # Montrer les 3 premiers
                logger.info(f"  • {event['start'].strftime('%b %d %H:%M')}")
            
            self.create_events(new_events)
        else:
            logger.warning("⚠️ Aucun événement futur dans le JSON")
        
        logger.info("=" * 50)
        logger.info("✅ Synchronisation terminée!")

def update_calendar():
    try:
        manager = PickleballCalendarManager()
        manager.sync_pickleball_events()
    
    except ValueError as e:
        logger.error(f"❌ Erreur config: {e}")
        logger.info("Vérifiez CALDAV_USER et CALDAV_PASSWORD dans .env")
    except Exception as e:
        logger.error(f"❌ Erreur: {e}")
        raise



2025-11-01 00:59:02,636 - __main__ - INFO - === STATUS DES AUTRES SECRETS (MASQUÉS) ===
2025-11-01 00:59:02,637 - __main__ - INFO - EMAIL: ✅ DÉFINI
2025-11-01 00:59:02,638 - __main__ - INFO - PASSWORD: ✅ DÉFINI
2025-11-01 00:59:02,639 - __main__ - INFO - MY_NAME: ✅ DÉFINI
2025-11-01 00:59:02,639 - __main__ - INFO - LOGON_URL: ✅ DÉFINI
2025-11-01 00:59:02,640 - __main__ - INFO - PLANNING_URL: ✅ DÉFINI
2025-11-01 00:59:02,642 - __main__ - INFO - LOGIN_URL: ✅ DÉFINI
2025-11-01 00:59:02,643 - __main__ - INFO - website_appointment: ✅ DÉFINI
2025-11-01 00:59:02,643 - __main__ - INFO - email_from: ✅ DÉFINI
2025-11-01 00:59:02,644 - __main__ - INFO - email_to: ✅ DÉFINI
2025-11-01 00:59:02,644 - __main__ - INFO - email_password: ✅ DÉFINI
2025-11-01 00:59:02,645 - __main__ - INFO - CALDAV_USER: ✅ DÉFINI
2025-11-01 00:59:02,647 - __main__ - INFO - CALDAV_PASSWORD: ✅ DÉFINI
2025-11-01 00:59:02,649 - __main__ - INFO - ✅ TOUS LES SECRETS SONT DÉFINIS
2025-11-01 00:59:02,650 - __main__ - DEBUG - Pour

In [5]:
DEFAULT_DATE = "03-Nov-25"
DEFAULT_TIME = "8:00"
DEFAULT_LEVEL = "Novice"

In [11]:
bot.quit()

2025-11-01 01:03:29,793 - selenium.webdriver.remote.remote_connection - DEBUG - DELETE http://localhost:35991/session/df71042cead742d4c888725f9f363618 {}
2025-11-01 01:03:29,861 - urllib3.connectionpool - DEBUG - http://localhost:35991 "DELETE /session/df71042cead742d4c888725f9f363618 HTTP/1.1" 200 0
2025-11-01 01:03:29,864 - selenium.webdriver.remote.remote_connection - DEBUG - Remote response: status=200 | data={"value":null} | headers=HTTPHeaderDict({'Content-Length': '14', 'Content-Type': 'application/json; charset=utf-8', 'cache-control': 'no-cache'})
2025-11-01 01:03:29,866 - selenium.webdriver.remote.remote_connection - DEBUG - Finished Request
2025-11-01 01:03:30,932 - __main__ - INFO - 🔒 Driver fermé


In [12]:
bot = SecureWebLogin(logon_url, email, password)
driver = bot.login()
# planning click on next depend de la date
driver.get(planning_url)
slot = bot.look_for_slots(DEFAULT_DATE, DEFAULT_LEVEL, DEFAULT_TIME)
#if slot
#bot.click_on_slot(slot)

2025-11-01 01:03:32,079 - __main__ - DEBUG - Initialisation du driver Chrome...
2025-11-01 01:03:32,082 - WDM - INFO - ====== WebDriver manager ======
2025-11-01 01:03:32,861 - WDM - INFO - Get LATEST chromedriver version for google-chrome
2025-11-01 01:03:32,868 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): googlechromelabs.github.io:443
2025-11-01 01:03:33,011 - urllib3.connectionpool - DEBUG - https://googlechromelabs.github.io:443 "GET /chrome-for-testing/latest-patch-versions-per-build.json HTTP/1.1" 200 13883
2025-11-01 01:03:33,031 - WDM - INFO - Get LATEST chromedriver version for google-chrome
2025-11-01 01:03:33,033 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): googlechromelabs.github.io:443
2025-11-01 01:03:33,173 - urllib3.connectionpool - DEBUG - https://googlechromelabs.github.io:443 "GET /chrome-for-testing/latest-patch-versions-per-build.json HTTP/1.1" 200 13883
2025-11-01 01:03:33,180 - WDM - INFO - Driver [/root/.wdm/dri

In [13]:
bot.click_on_slot(slot)

2025-11-01 01:03:41,853 - selenium.webdriver.remote.remote_connection - DEBUG - POST http://localhost:48225/session/994f21b1b5ef9a98b772a8bdf7438fec/element/f.DBBD9CD460B83F2A11029A0832DE6247.d.3ECB31E75473AF72429075FF6E54998C.e.86/element {'using': 'xpath', 'value': './..'}


2025-11-01 01:03:41,897 - urllib3.connectionpool - DEBUG - http://localhost:48225 "POST /session/994f21b1b5ef9a98b772a8bdf7438fec/element/f.DBBD9CD460B83F2A11029A0832DE6247.d.3ECB31E75473AF72429075FF6E54998C.e.86/element HTTP/1.1" 200 0
2025-11-01 01:03:41,898 - selenium.webdriver.remote.remote_connection - DEBUG - Remote response: status=200 | data={"value":{"element-6066-11e4-a52e-4f735466cecf":"f.DBBD9CD460B83F2A11029A0832DE6247.d.3ECB31E75473AF72429075FF6E54998C.e.131"}} | headers=HTTPHeaderDict({'Content-Length': '127', 'Content-Type': 'application/json; charset=utf-8', 'cache-control': 'no-cache'})
2025-11-01 01:03:41,900 - selenium.webdriver.remote.remote_connection - DEBUG - Finished Request
2025-11-01 01:03:41,901 - selenium.webdriver.remote.remote_connection - DEBUG - POST http://localhost:48225/session/994f21b1b5ef9a98b772a8bdf7438fec/element/f.DBBD9CD460B83F2A11029A0832DE6247.d.3ECB31E75473AF72429075FF6E54998C.e.131/elements {'using': 'xpath', 'value': ".//*[contains(text()

True

In [14]:
bot.get_driver().save_screenshot("5-test.png")
bot.click_for_me('Serge Lebon')



2025-11-01 01:03:47,293 - selenium.webdriver.remote.remote_connection - DEBUG - GET http://localhost:48225/session/994f21b1b5ef9a98b772a8bdf7438fec/screenshot {}
2025-11-01 01:03:47,500 - urllib3.connectionpool - DEBUG - http://localhost:48225 "GET /session/994f21b1b5ef9a98b772a8bdf7438fec/screenshot HTTP/1.1" 200 0
2025-11-01 01:03:47,501 - selenium.webdriver.remote.remote_connection - DEBUG - Remote response: status=200 | data={"value":"..."} | headers=HTTPHeaderDict({'Content-Length': '170676', 'Content-Type': 'application/json; charset=utf-8', 'cache-control': 'no-cache'})
2025-11-01 01:03:47,507 - selenium.webdriver.remote.remote_connection - DEBUG - Finished Request
2025-11-01 01:03:47,513 - selenium.webdriver.remote.remote_connection - DEBUG - POST http://localhost:48225/session/994f21b1b5ef9a98b772a8bdf7438fec/elements {'using': 'xpath', 'value': "//*[contains(text(), 'Serge Lebon')]"}
2025-11-01 01:03:47,595 - urllib3.connectionpool - DEBUG - http://localhost:48225 "POST /sess

ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=141.0.7390.122); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#elementnotinteractableexception
Stacktrace:
#0 0x555555f8bfea <unknown>
#1 0x555555a09b4e <unknown>
#2 0x555555a5d11e <unknown>
#3 0x555555a5016b <unknown>
#4 0x555555a81e12 <unknown>
#5 0x555555a4fae7 <unknown>
#6 0x555555a81fde <unknown>
#7 0x555555aa7b3b <unknown>
#8 0x555555a81be3 <unknown>
#9 0x555555a4e2d2 <unknown>
#10 0x555555a4ef91 <unknown>
#11 0x555555f4f9e8 <unknown>
#12 0x555555f5384f <unknown>
#13 0x555555f36ec9 <unknown>
#14 0x555555f543f5 <unknown>
#15 0x555555f1c74f <unknown>
#16 0x555555f78cc8 <unknown>
#17 0x555555f78ea3 <unknown>
#18 0x555555f8af83 <unknown>
#19 0x7fffff1d3b7b <unknown>


In [ ]:
#secure_login = SecureWebLogin(logon_url, email, password)
#secure_login.login()  # ✅ D'abord se connecter
#secure_login.save_appointments_json()
#secure_login.quit()
#send_all_appointments_email(load_appointments())
#update_calendar()

bot = SecureWebLogin(logon_url, email, password)
driver = bot.login()
# planning click on next depend de la date
driver.get(planning_url)
slot = bot.look_for_slots(DEFAULT_DATE, DEFAULT_LEVEL, DEFAULT_TIME)
#if slot
bot.click_on_slot(slot)
bot.click_for_me('Serge Lebon')
bot.get_driver().save_screenshot("5-test.png")
bot.click()
bot.get_driver().save_screenshot("6-test.png")
bot.click_confirm_basket()

2025-11-01 00:00:52,253 - __main__ - DEBUG - Initialisation du driver Chrome...
2025-11-01 00:00:52,259 - WDM - INFO - ====== WebDriver manager ======
2025-11-01 00:00:52,814 - WDM - INFO - Get LATEST chromedriver version for google-chrome
2025-11-01 00:00:52,831 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): googlechromelabs.github.io:443
2025-11-01 00:00:53,062 - urllib3.connectionpool - DEBUG - https://googlechromelabs.github.io:443 "GET /chrome-for-testing/latest-patch-versions-per-build.json HTTP/1.1" 200 13883
2025-11-01 00:00:53,171 - WDM - INFO - Get LATEST chromedriver version for google-chrome
2025-11-01 00:00:53,173 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): googlechromelabs.github.io:443
2025-11-01 00:00:53,315 - urllib3.connectionpool - DEBUG - https://googlechromelabs.github.io:443 "GET /chrome-for-testing/latest-patch-versions-per-build.json HTTP/1.1" 200 13883
2025-11-01 00:00:53,332 - WDM - INFO - Get LATEST chromedrive

True

In [4]:
bot.click_on_slot(slot)
bot.get_driver().save_screenshot("4-test.png")

2025-10-31 23:56:28,971 - selenium.webdriver.remote.remote_connection - DEBUG - POST http://localhost:42427/session/d10dc603bc3a794d5f33c1eaa28bde36/element/f.2A173E3D920688D47262CFCA2CFEFECF.d.66B1E0837EB581CB42093BC6DC1E1A65.e.86/element {'using': 'xpath', 'value': './..'}
2025-10-31 23:56:29,039 - urllib3.connectionpool - DEBUG - http://localhost:42427 "POST /session/d10dc603bc3a794d5f33c1eaa28bde36/element/f.2A173E3D920688D47262CFCA2CFEFECF.d.66B1E0837EB581CB42093BC6DC1E1A65.e.86/element HTTP/1.1" 200 0
2025-10-31 23:56:29,040 - selenium.webdriver.remote.remote_connection - DEBUG - Remote response: status=200 | data={"value":{"element-6066-11e4-a52e-4f735466cecf":"f.2A173E3D920688D47262CFCA2CFEFECF.d.66B1E0837EB581CB42093BC6DC1E1A65.e.131"}} | headers=HTTPHeaderDict({'Content-Length': '127', 'Content-Type': 'application/json; charset=utf-8', 'cache-control': 'no-cache'})
2025-10-31 23:56:29,040 - selenium.webdriver.remote.remote_connection - DEBUG - Finished Request
2025-10-31 23:5

True

In [9]:
DEFAULT_DATE = "03-Nov-25"
DEFAULT_TIME = "8:00"
DEFAULT_LEVEL = "Novice"

In [8]:
#secure_login.get_driver().save_screenshot("3-test.png")
secure_login.go_url(planning_url)
secure_login.get_driver().save_screenshot("4-test.png")


2025-10-31 23:17:09,680 - __main__ - INFO - 🔗 Navigation vers https://mycalgaryjcc.com/CJCC/public/category/browse/PICKOPEN


True

In [ ]:
logger.info("🔍 Recherche de boutons avec données de cours...")
course_buttons = secure_login.get_driver().find_elements(By.XPATH, "//button[@data-class-time]")

2025-10-31 23:22:42,782 - root - INFO - 🔍 Recherche de boutons avec données de cours...


In [ ]:

print(f"📋 {len(course_buttons)} bouton(s) de cours trouvé(s)")
found_slots = None
for i, button in enumerate(course_buttons):
    
        # Extraire les données du bouton
        class_name = button.get_attribute('data-class-name') or ''
        class_date = button.get_attribute('data-class-date') or ''
        class_time = button.get_attribute('data-class-time') or ''
        class_spaces = button.get_attribute('data-class-spaces') or '0'
        class_location = button.get_attribute('data-class-location') or ''
        class_venue = button.get_attribute('data-class-venue') or ''
        logger.debug(class_name)
        logger.debug(class_date)




📋 44 bouton(s) de cours trouvé(s)
2025-10-31 23:23:05,159 - root - INFO - 90 Min Pickleball Intermediate
2025-10-31 23:23:05,199 - root - INFO - 90 Minute Pickleball - Novice
2025-10-31 23:23:05,231 - root - INFO - 90 Min Pickleball Intermediate
2025-10-31 23:23:05,255 - root - INFO - 90 Minute Pickleball - Novice
2025-10-31 23:23:05,280 - root - INFO - 90 Minute Pickleball - Novice
2025-10-31 23:23:05,304 - root - INFO - 90 Min Pickleball Intermediate
2025-10-31 23:23:05,328 - root - INFO - 90 Min Pickleball Intermediate
2025-10-31 23:23:05,353 - root - INFO - 90 Minute Pickleball - Novice
2025-10-31 23:23:05,378 - root - INFO - Indoor Pickleball Novice
2025-10-31 23:23:05,402 - root - INFO - Indoor Pickleball Intermediate
2025-10-31 23:23:05,427 - root - INFO - Indoor Pickleball Novice
2025-10-31 23:23:05,451 - root - INFO - Indoor Pickleball Intermediate
2025-10-31 23:23:05,474 - root - INFO - Indoor Pickleball  Advanced
2025-10-31 23:23:05,498 - root - INFO - Indoor Pickleball Inte

In [ ]:
ef look_for_slots(driver, target_date, target_level, target_time):
    print("🔍 Recherche de boutons avec données de cours...")

    # Chercher tous les boutons avec data-class-time (structure de votre site)
    course_buttons = driver.find_elements(By.XPATH, "//button[@data-class-time]")

    print(f"📋 {len(course_buttons)} bouton(s) de cours trouvé(s)")

    found_slots = None

    for i, button in enumerate(course_buttons):
        
            # Extraire les données du bouton
            class_name = button.get_attribute('data-class-name') or ''
            class_date = button.get_attribute('data-class-date') or ''
            class_time = button.get_attribute('data-class-time') or ''
            class_spaces = button.get_attribute('data-class-spaces') or '0'
            class_location = button.get_attribute('data-class-location') or ''
            class_venue = button.get_attribute('data-class-venue') or ''
            
            #print(f"\n  Cours {i+1}:")
            #print(f"    Nom: {class_name}")
            #print(f"    Date: {class_date}")
            #print(f"    Heure: {class_time}")
            #print(f"    Places: {class_spaces}")
            
            # Vérifier si c'est le bon jour ET contient 6:30 PM
            date_match = target_date in class_date
            level_match = target_level in class_name
            time_match = target_time in class_time.split("-")[0] or target_time.lstrip() in class_time.split("-")[0]
            
            if date_match:
                 pass
                #print(f"    Heure: {class_time}")
                #print(f"    Nom: {class_name}")
                #print(f"    Date: {class_date}")
                #print(f"    Heure: {class_time}")
                #print(f"    Places: {class_spaces}")
            
            if date_match and time_match and level_match:
                print(f"    ✅ CORRESPONDANCE TROUVÉE!")
                spaces_available = int(class_spaces)
                print(spaces_available)
                #driver.save_screenshot("7_find_date_planning.png")
                slot_info = {
                            'button': button,
                            'class_name': class_name,
                            'class_date': class_date,
                            'class_time': class_time,
                            'class_spaces': spaces_available,
                            'class_location': class_location,
                            'class_venue': class_venue,
                            'available': spaces_available > 0
                        }
                        
                found_slots = slot_info
                        
                if spaces_available > 0:
                    print(f"    ✅ DISPONIBLE - {spaces_available} place(s)")
                    
                else:
                    print(f"    ❌ COMPLET - 0 place")
                break
    if found_slots == None:
      